# Australian Suburb and Town recommendation engine



The first step is to import standard libraries

In [1]:
!pip install pandasdmx

    100% |████████████████████████████████| 51kB 13.9MB/s ta 0:00:01
  Running setup.py bdist_wheel for jsonpath-rw ... done
  Stored in directory: /home/dsxuser/.cache/pip/wheels/5c/00/9a/82822db383c2d96dcebf839786665a185f92d37e5026f9806f
Successfully built jsonpath-rw
tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.


In [6]:
# install and import the folium library to visualise the neighbourhood clusters on map
!pip install folium

In [2]:
import pandas as pd
import numpy as np
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import requests
from sklearn.cluster import KMeans
from pandasdmx import Request
from bs4 import BeautifulSoup

In [7]:
import folium # map rendering library

ImportError: No module named 'folium'

## Get suburb and postcode data
List of Australian postcodes and suburbs are available online on many websites. I've replicated the file from http://www.corra.com.au/australian-postcode-location-data/ on Github


In [8]:
filename = "https://raw.githubusercontent.com/DataistDogma/Coursera_Capstone/master/Australian_Post_Codes_Lat_Lon/Australian_Post_Codes_Lat_Lon.csv"
postcodes = pd.read_csv(filename)
print("Postcodes shape: ", postcodes.shape)
postcodes.head()

Postcodes shape:  (16080, 7)


,postcode,suburb,state,dc,type,lat,lon
0,200,AUSTRALIAN NATIONAL UNIVERSITY,ACT,AUSTRALIAN NATIONAL UNI LPO,Post Office Boxes,-35.277272,149.117136
1,221,BARTON,ACT,NaN,LVR,-35.201372,149.095065
2,800,DARWIN,NT,DARWIN DELIVERY CENTRE,Delivery Area,-12.801028,130.955789
3,801,DARWIN,NT,DARWIN DELIVERY CENTRE,Post Office Boxes,-12.801028,130.955789
4,804,PARAP,NT,PARAP,Post Office Boxes,-12.432181,130.843310


We can see that each row represents a Distribution Centre (dc) rather than a suburb or postcode. We're looking for a list of unique suburbs, so we'll filter the dataframe for unique suburbs and remove the 'dc' and 'type' column, which we don't need

In [9]:
postcodes.drop_duplicates(subset='suburb', inplace=True)
postcodes.drop(['dc', 'type'],axis=1,inplace=True )
postcodes.shape

(14270, 5)

The code below checks for null values in each column of the dataframe. It looks like there is just one null in each column:

In [10]:
missing_data = postcodes.isnull()
for column in missing_data.columns.values.tolist():
    print(column)
    print (missing_data[column].value_counts())
    print("")    

postcode
False    14270
Name: postcode, dtype: int64

suburb
False    14269
True         1
Name: suburb, dtype: int64

state
False    14269
True         1
Name: state, dtype: int64

lat
False    14269
True         1
Name: lat, dtype: int64

lon
False    14269
True         1
Name: lon, dtype: int64



We'll take a look at the row in question:

In [11]:
postcodes[postcodes['suburb'].isnull()]

,postcode,suburb,state,lat,lon
16079,0,NaN,NaN,NaN,NaN


It's just the last row of the original csv with no data, so we'll drop it and reset the index

In [12]:
postcodes.dropna(inplace=True)
postcodes.reset_index(inplace=True, drop=True)
postcodes.head()

,postcode,suburb,state,lat,lon
0,200,AUSTRALIAN NATIONAL UNIVERSITY,ACT,-35.277272,149.117136
1,221,BARTON,ACT,-35.201372,149.095065
2,800,DARWIN,NT,-12.801028,130.955789
3,804,PARAP,NT,-12.432181,130.843310
4,810,ALAWA,NT,-12.378451,130.877014


For the sake of testing the code throughout development, we'll also create a smaller subset of suburbs, being those in Metropolitan Melbourne, the second-largest city in Australia. The postcode range for Melbourne are postcode between 3000-3207 & 8000-8499

In [13]:
melb_postcodes = postcodes[(postcodes['postcode']>=3000)&(postcodes['postcode']<=3207)|(postcodes['postcode']>=8000)&(postcodes['postcode']<=8499)]
melb_postcodes.head()

,postcode,suburb,state,lat,lon
4901,3000,MELBOURNE,VIC,-37.814563,144.970267
4902,3002,EAST MELBOURNE,VIC,-37.816640,144.987811
4903,3003,WEST MELBOURNE,VIC,-37.806255,144.941123
4904,3005,WORLD TRADE CENTRE,VIC,-37.822262,144.954856
4905,3006,SOUTHBANK,VIC,-37.823258,144.965926


## Assign venue clusters
Use the Foursquare API to get venue recommendations for each suburb and then use K-means to assign a venue cluster to the suburb based on the recommendations returned

### Get venue recommendations for the suburbs

In [14]:
#intialise Foursquare API credentials
CLIENT_ID = 'EKC0OWGJC1SY1AE1UHB4PUPH2JGARZTQK1U5C1USTUNA43JF' # your Foursquare ID
CLIENT_SECRET = 'CHH0EPUEN2PH4WBDV4XHTDT5NWTUZ2SYVIQALUOZXWYNABRO' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [15]:
#Use the function to extract the category from the dataframe (because the column name could be either 'categories' or 'venue.categories')
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [16]:
#define a function to return all the venues for a given neighborhood

#pass the neighborhood (nb), the latitude (la), the radius and the limit 
def get_venues(nb, la, lo, radius, limit):
    
    #print(nb)
    
    #form the request url and request only the items (which are the venues)
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, la, lo, VERSION, limit)
    items = requests.get(url).json()["response"]['groups'][0]['items']
    
    #include exception handling where Foursquare request fails for a particular Neighbourhood
    if items == []:
        print(" -- Foursquare request for ",nb,"returned no results --")
        return None
    else:
        # flatten JSON, filter for only wanted columns then use the get_category_type funtion to replace the category list with just the category.
        venues1 = json_normalize(items)
        filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
        venues1 =venues1.loc[:, filtered_columns]
        venues1['venue.categories'] = venues1.apply(get_category_type, axis=1)

        #define a new dataframe with the Neighbourhood information
        venues2 = pd.DataFrame(columns=['Suburb','Suburb Latitude', 
                      'Suburb Longitude']) 

        #for each of the venues returned, add the venue dataframe to the (empty) neighbourhood dataframe and fill all rows in the the neighbourhood columns with the neighborhood name, the neighbourhood latitude and longitude
        i=0
        for row in venues1:
            venues2[row]=venues1[row]
            venues2['Suburb']=nb
            venues2['Suburb Latitude']=la
            venues2['Suburb Longitude']=lo
            i=i+1

        #rename the columns
        venues2.rename(index=str,columns={"venue.name":"Venue","venue.categories":"Venue Category","venue.location.lat":"Venue Latitude","venue.location.lng":"Venue Longitude"}, inplace=True)
    
    return venues2

In [17]:
#test out the get_venues function
row = 300
n = postcodes.loc[row]['suburb']
n_lat = postcodes.loc[row]['lat']
n_long = postcodes.loc[row]['lon']

df2=get_venues(n, n_lat, n_long, 500, 100)
df2.head()

OSError: Could not find a suitable TLS CA certificate bundle, invalid path: /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/certifi/cacert.pem

In [20]:
#Iterate thorough all the suburbs of the dataframe df, adding each set of venues to the df_suburb_venues dataframe
df=melb_postcodes
df_suburb_venues=pd.DataFrame()
radius = 500
limit = 100
print("Getting venue recommendations, please wait......")
for i, row in enumerate(df['suburb']):
        df_suburb_venues=df_suburb_venues.append(get_venues(df.iloc[i]['suburb'], df.iloc[i]['lat'], df.iloc[i]['lon'], radius, limit),ignore_index=True)
print("...... Complete")

Getting venue recommendations, please wait......


OSError: Could not find a suitable TLS CA certificate bundle, invalid path: /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/certifi/cacert.pem

In [24]:
df_suburb_venues.head()

,Suburb,Suburb Latitude,Suburb Longitude,Venue,Venue Category,Venue Latitude,Venue Longitude
0,MELBOURNE,-37.814563,144.970267,Eau de Vie,Speakeasy,-37.815748,144.971542
1,MELBOURNE,-37.814563,144.970267,Chin Chin,Asian Restaurant,-37.815451,144.970189
2,MELBOURNE,-37.814563,144.970267,Coda,Vietnamese Restaurant,-37.815625,144.969906
3,MELBOURNE,-37.814563,144.970267,Grand Hyatt Melbourne,Hotel,-37.815213,144.969658
4,MELBOURNE,-37.814563,144.970267,Pastuso,Peruvian Restaurant,-37.815354,144.971062


In [25]:
counts = df_suburb_venues['Venue Category'].value_counts()
counts

Café                            6143
Fast Food Restaurant            1762
Supermarket                     1424
Grocery Store                   1421
Sandwich Place                  1024
Coffee Shop                      914
Park                             829
Convenience Store                781
Shopping Mall                    768
Pizza Place                      756
Pub                              725
Gym                              664
Portuguese Restaurant            645
Bakery                           633
Bar                              591
Burger Joint                     584
Electronics Store                581
Vietnamese Restaurant            579
Department Store                 566
Italian Restaurant               560
Japanese Restaurant              535
Thai Restaurant                  468
Chinese Restaurant               419
Malay Restaurant                 408
Train Station                    360
Indian Restaurant                352
Liquor Store                     346
M

Interesting to see 'Portugese Restaurant' feature so highly on the above list. I suspect no Melbournian would have expected that, as we don't really have much of a Portugese community and I've personally never seen a Portugese restaurant. Vietnamese, Italian and Japanese are surely more prevalent! Looking more closely it turns out that this is basically due to the prevalence of Nando's fast food restaurants: 

In [26]:
df_suburb_venues[df_suburb_venues['Venue Category']=='Portuguese Restaurant'].head()

,Suburb,Suburb Latitude,Suburb Longitude,Venue,Venue Category,Venue Latitude,Venue Longitude
237,WEST MELBOURNE,-37.806255,144.941123,Nando's,Portuguese Restaurant,-37.812157,144.938183
586,DOCKLANDS,-37.814719,144.948039,Nando's,Portuguese Restaurant,-37.812157,144.938183
1908,WILLIAMSTOWN NORTH,-37.857681,144.887041,Nando's,Portuguese Restaurant,-37.827833,144.847661
1999,ALTONA,-37.869275,144.830286,Nando's,Portuguese Restaurant,-37.861810,144.902400
2015,ALTONA,-37.869275,144.830286,Nando's,Portuguese Restaurant,-37.823350,144.823810


In [27]:
df_onehot = pd.get_dummies(df_suburb_venues['Venue Category'])
#df_onehot.drop('suburb',axis=1,inplace=True)
#insert a new column for the Neighbourhood values (note that immediately reusing the 'Neighborhood' name causes as error as you can't add a Neighourhood column that already exists, so renamed the column post the insert)
df_onehot.insert(0,"New",df_suburb_venues['Suburb'])
df_onehot.rename(columns={'New':'Suburb'}, inplace=True)

#get the mean frequency occurance
df_onehot = df_onehot.groupby("Suburb").mean().reset_index()
df_onehot.head()

,Suburb,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Whisky Bar,Wine Bar,Wine Shop,Winery,Women's Store,Xinjiang Restaurant,Yoga Studio,Yunnan Restaurant,Zoo,Zoo Exhibit
0,ABECKETT STREET,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.01,0.0,...,0.0,0.010000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ABERFELDIE,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,...,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,AIRPORT WEST,0.0,0.000000,0.0,0.011765,0.0,0.0,0.0,0.00,0.0,...,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ALBANVALE,0.0,0.010638,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,...,0.0,0.010638,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ALBERT PARK,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,...,0.0,0.010000,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
#put the top10 into a dataframe
df_topvenues=pd.DataFrame(columns=['Suburb','#1','#2','#3','#4','#5','#6','#7','#8','#9','#10'])

for row in range(0,len(df_onehot)):
    suburb = df_onehot.iloc[row][0]
    topten = df_onehot.iloc[row][1:].sort_values(ascending=False).head(10)
    temp=pd.DataFrame([[suburb,topten.index[0],topten.index[1],topten.index[2],topten.index[3],topten.index[4],topten.index[5],topten.index[6],topten.index[7],topten.index[8],topten.index[9]]],columns=['Suburb','#1','#2','#3','#4','#5','#6','#7','#8','#9','#10'])
    df_topvenues=df_topvenues.append(temp)
df_topvenues.reset_index(inplace=True, drop=True)
df_topvenues.head()

,Suburb,#1,#2,#3,#4,#5,#6,#7,#8,#9,#10
0,ABECKETT STREET,Korean Restaurant,Café,Coffee Shop,Japanese Restaurant,Dessert Shop,Indonesian Restaurant,Bubble Tea Shop,Cosmetics Shop,Cocktail Bar,Clothing Store
1,ABERFELDIE,Café,Electronics Store,Grocery Store,Japanese Restaurant,Bakery,Pizza Place,Pub,Coffee Shop,Shopping Mall,Gym
2,AIRPORT WEST,Fast Food Restaurant,Café,Supermarket,Sandwich Place,Grocery Store,Portuguese Restaurant,Shopping Mall,Italian Restaurant,Convenience Store,Electronics Store
3,ALBANVALE,Fast Food Restaurant,Portuguese Restaurant,Café,Shopping Mall,Supermarket,Grocery Store,Vietnamese Restaurant,Chinese Restaurant,Sandwich Place,Electronics Store
4,ALBERT PARK,Café,Gastropub,Coffee Shop,Pub,Japanese Restaurant,Burger Joint,Beach,Breakfast Spot,Pet Store,Middle Eastern Restaurant


### Assign a cluster based on the recommendations using the k-means clustering algorithm from Scikit Learn

In [29]:
# import k-means
from sklearn.cluster import KMeans

df_clustering = df_onehot.drop('Suburb',axis=1)

# set number of clusters
k = 5

# run k-means clustering
kmeans = KMeans(n_clusters=k, random_state=0).fit(df_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:100]

array([0, 3, 4, 4, 3, 4, 3, 1, 3, 3, 2, 3, 4, 3, 3, 1, 3, 3, 1, 1, 2, 4, 3,
       4, 3, 3, 3, 3, 2, 1, 1, 2, 2, 3, 2, 1, 1, 1, 1, 1, 2, 1, 1, 3, 1, 3,
       3, 1, 1, 1, 2, 2, 1, 1, 3, 1, 1, 1, 1, 1, 3, 3, 3, 3, 4, 0, 0, 0, 3,
       0, 0, 1, 1, 3, 3, 4, 4, 1, 4, 4, 3, 3, 3, 3, 2, 0, 0, 1, 4, 2, 2, 3,
       3, 2, 3, 3, 3, 1, 1, 3], dtype=int32)

In [30]:
df_topvenues.insert(0,'Venues Cluster',kmeans.labels_)

In [31]:
df_topvenues[df_topvenues['Venues Cluster']==2]

,Venues Cluster,Suburb,#1,#2,#3,#4,#5,#6,#7,#8,#9,#10
10,2,ALTONA MEADOWS,Fast Food Restaurant,Supermarket,Café,Portuguese Restaurant,Sandwich Place,Bar,Gym,Grocery Store,Pizza Place,Electronics Store
20,2,ASPENDALE GARDENS,Café,Beach,Grocery Store,Supermarket,Fast Food Restaurant,Golf Course,Train Station,Convenience Store,Pub,Liquor Store
28,2,BANGHOLME,Fast Food Restaurant,Supermarket,Café,Grocery Store,Beach,Portuguese Restaurant,Convenience Store,Sandwich Place,Train Station,Golf Course
31,2,BAYSWATER,Fast Food Restaurant,Malay Restaurant,Department Store,Shopping Mall,Gym,Grocery Store,Golf Course,Café,Coffee Shop,Japanese Restaurant
32,2,BAYSWATER NORTH,Malay Restaurant,Fast Food Restaurant,Grocery Store,Golf Course,Japanese Restaurant,Shopping Mall,Athletics & Sports,Department Store,Supermarket,Coffee Shop
34,2,BEDFORD ROAD,Café,Shopping Mall,Grocery Store,Malay Restaurant,Japanese Restaurant,Coffee Shop,Movie Theater,Pub,Fast Food Restaurant,Supermarket
40,2,BELVEDERE PARK,Beach,Café,Supermarket,Fast Food Restaurant,Convenience Store,Department Store,Gas Station,Grocery Store,Sandwich Place,Shopping Mall
50,2,BONBEACH,Café,Fast Food Restaurant,Supermarket,Beach,Grocery Store,Portuguese Restaurant,Convenience Store,Sandwich Place,Gas Station,Pizza Place
51,2,BORONIA,Grocery Store,Café,Malay Restaurant,Gym,Convenience Store,Fast Food Restaurant,Supermarket,Shopping Mall,Sandwich Place,Burger Joint
84,2,CAMPBELLFIELD,Fast Food Restaurant,Grocery Store,Café,Middle Eastern Restaurant,Shopping Mall,Supermarket,Gym,Convenience Store,Sandwich Place,Electronics Store


In [32]:
df_topvenues[df_topvenues['Suburb']=='BRUNSWICK WEST']

,Venues Cluster,Suburb,#1,#2,#3,#4,#5,#6,#7,#8,#9,#10
70,0,BRUNSWICK WEST,Café,Bar,Middle Eastern Restaurant,Grocery Store,Beer Garden,Park,Thai Restaurant,Ice Cream Shop,Indian Restaurant,Pub


In [34]:
df_merged = postcodes

df_merged = df_merged.join(df_topvenues.set_index('Suburb'), on='suburb')

#drop rows with no clusters
df_melbourne_merged = df_merged.dropna(subset=['Venues Cluster'])
df_melbourne_merged.head()


,postcode,suburb,state,lat,lon,Venues Cluster,#1,#2,#3,#4,#5,#6,#7,#8,#9,#10
4901,3000,MELBOURNE,VIC,-37.814563,144.970267,0.0,Coffee Shop,Cocktail Bar,Italian Restaurant,Café,Hotel,Asian Restaurant,Japanese Restaurant,Bar,BBQ Joint,Bookstore
4902,3002,EAST MELBOURNE,VIC,-37.816640,144.987811,0.0,Café,Vietnamese Restaurant,Cricket Ground,Park,Tennis Stadium,Bakery,Vegetarian / Vegan Restaurant,Wine Bar,Football Stadium,Breakfast Spot
4903,3003,WEST MELBOURNE,VIC,-37.806255,144.941123,0.0,Café,Korean Restaurant,Bar,Pub,Hotel,Sandwich Place,Park,Restaurant,Burger Joint,Italian Restaurant
4904,3005,WORLD TRADE CENTRE,VIC,-37.822262,144.954856,0.0,Café,Japanese Restaurant,Coffee Shop,Hotel,Bar,Australian Restaurant,Italian Restaurant,Burger Joint,Steakhouse,French Restaurant
4905,3006,SOUTHBANK,VIC,-37.823258,144.965926,0.0,Hotel,Bar,Theater,Italian Restaurant,Performing Arts Venue,Park,Art Gallery,Plaza,Café,Burger Joint


In [35]:
latitude=postcodes[postcodes['suburb']=='MELBOURNE'].lat
longitude=postcodes[postcodes['suburb']=='MELBOURNE'].lon
print(latitude,longitude)

4901   -37.814563
Name: lat, dtype: float64 4901    144.970267
Name: lon, dtype: float64


In [36]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude])

colors_list = ['Red','Blue','Green','Yellow','Purple','Pink','Orange']

# add markers to the map
#markers_colors = []
#for lat, lon, poi, cluster in zip(df_melbourne_merged['lat'], df_melbourne_merged['lon'], df_melbourne_merged['suburb'], df_melbourne_merged['Cluster Labels']):
#    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
#    folium.CircleMarker(
#        [lat, lon],
#        radius=5,
#        popup=label,
#        color=colors_list[int(cluster)],
#        fill=True,
#       fill_opacity=0.7).add_to(map_clusters)

       
map_clusters

ImportError: No module named 'distutils.sysconfig'

# Assign cultural clusters
Use the Australian Beaureu of Statistics API to get a breakdown on the ancestry for the suburb suburb and then use K-means to assign a venue cluster suburbs with similar ancestry

In [39]:
mp_url= 'ABS_CENSUS2011_T09/TOT+1+2+3+4+Z.TOT+TOTP+1101+1102+6101+3204+2303+2101+5201+2305+2306+3205+3304+7106+2201+3103+6902+4106+3206+3104+1201+1202+3307+3308+2102+3213+7115+9215+3106+4907+5107+2103+OTH+Z.2.SA2.206031116.A/all?detail=Full&dimensionAtObservation=AllDimensions'
bw_url= 'ABS_CENSUS2011_T09/TOT+1+2+3+4+Z.TOT+TOTP+1101+1102+6101+3204+2303+2101+5201+2305+2306+3205+3304+7106+2201+3103+6902+4106+3206+3104+1201+1202+3307+3308+2102+3213+7115+9215+3106+4907+5107+2103+OTH+Z.2.SA2.206011105.A/all?detail=Full&dimensionAtObservation=AllDimensions'
allvic_url='ABS_CENSUS2011_T09/TOT+1+2+3+4+Z.TOT+TOTP+1101+1102+6101+3204+2303+2101+5201+2305+2306+3205+3304+7106+2201+3103+6902+4106+3206+3104+1201+1202+3307+3308+2102+3213+7115+9215+3106+4907+5107+2103+OTH+Z.2.SA2..A/all?detail=Full&dimensionAtObservation=AllDimensions'

In [40]:
#create request object
abs = Request('ABS')
print(abs,dir(abs))

NameError: name 'Request' is not defined

In [ ]:
#create response object
response = abs.data(allvic_url)
print(response,dir(response))

In [ ]:
#create dataset object
dataset=response.data
print(dataset, dir(dataset))

In [ ]:
##create top level series
data=response.write()
print(type(data),'\n',data, dir(data))

The data above is arranged in a single multi index column (with region being the postcode code and ANCP being the ancestry country code), so we need to pull the indexes into columns and then we can access the columns we need more easily:

In [ ]:

df_ancestry = data.reset_index(name='Value')
print(df_ancestry.shape)
df_ancestry.head()

In [ ]:
#Take only 2011 data
df_ancestry = df_ancestry[(df_ancestry['TIME_PERIOD']=='2011')]

#The data includes six different measures of ancestry (father only, mother only, both).
#in this case the only measure i'm interested in the totals for each ancestry
df_ancestry = df_ancestry[(df_ancestry['MEASURE']=='TOT')&(df_ancestry['ANCP']!='TOT')]

#pivot the Ancestry to columns in preparation for k-means analysis
df_ancestry = df_ancestry.pivot(index='REGION',columns='ANCP', values='Value')

df_ancestry.head()

In [ ]:
# set number of clusters
k = 10

# run k-means clustering
kmeans = KMeans(n_clusters=k, random_state=0).fit(df_ancestry)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:100]

In [ ]:
df_ancestry.insert(0,'Ancestry Cluster',kmeans.labels_)
df_ancestry.head()

Now to figure out how to get the Countries related to each of the country codes....

In [ ]:
import xml.etree.ElementTree as ET

In [ ]:
user_agent_url = 'http://stat.data.abs.gov.au/restsdmx/sdmx.ashx/GetDataStructure/ABS_CENSUS2011_T09'
xml_data = requests.get(user_agent_url).content

In [ ]:
tree = ET.fromstring(xml_data)
tree.attrib

In [ ]:
#Create a dataframe which will include all the codes
df_codes = pd.DataFrame(columns=['code','value'])

#parse the structure xml to get the codes:
for child in tree:
    for lower in child:
        for codelist in lower:
            for description in codelist:
                if(description.attrib.get('{http://www.w3.org/XML/1998/namespace}lang')=='en'):
                    code = codelist.attrib.get('value')
                    value = description.text
                    temp_df = pd.DataFrame([[code,value]],columns=['code','value'])
                    df_codes=pd.concat([df_codes,temp_df])

#set the code as the index of the table
df_codes.set_index('code', inplace=True)

#strip whitespaces from the values column
df_codes['value'] = df_codes['value'].str.strip()
df_codes.head()

In [ ]:
df_codes.loc['1102']

In [ ]:
df_codes[df_codes['value']=='Vietnamese']

In [ ]:
df_ancestry.head()

In [ ]:
df_ancestry = df_ancestry.join(df_codes)

In [ ]:
df_ancestry[df_ancestry['Ancestry Cluster']==8].head()

In [ ]:
df_ancestry[df_ancestry['value']=='Footscray'].max(axis=1)

In [ ]:
# select only the required columns, fix the names and make suburb lowercase for searching
df_ancestry = df_ancestry[['value','Ancestry Cluster']]
df_ancestry.rename(columns={"value":"suburb"}, inplace=True)


In [ ]:

#strip out whitespace, change to lowercase
df_ancestry['suburb'] = df_ancestry['suburb'].str.lower()
df_melbourne_merged['suburb']=df_melbourne_merged['suburb'].str.lower()
#df_melbourne_merged['suburb']=df_melbourne_merged['suburb'].str.strip()
#df_melbourne_merged['suburb']=df_melbourne_merged['suburb'].str.replace(" ", "_")
#df_ancestry['suburb']=df_ancestry['suburb'].str.strip()
#df_ancestry['suburb']=df_ancestry['suburb'].str.replace(" ", "_")

#merge on suburb
df_complete = df_melbourne_merged.merge(df_ancestry, how='left', on='suburb')

In [ ]:
#merge on suburb? only matches about half the suburbs!
df_complete = df_melbourne_merged.merge(df_ancestry, how='left', on='suburb')
df_complete

In [ ]:
df_complete[pd.isnull(df_complete['Ancestry Cluster'])]
    

In [ ]:
for suburb in df_complete[pd.isnull(df_complete['Ancestry Cluster'])]['suburb']:
    print(df_ancestry.loc[df_ancestry['suburb'].str.contains(suburb)]['Ancestry Cluster'])

In [ ]:
for suburb in df_complete[pd.isnull(df_complete['Ancestry Cluster'])]['suburb']:
    if(len(df_ancestry.loc[df_ancestry['suburb'].str.contains(suburb)]['Ancestry Cluster'])>0):
        print(df_ancestry.loc[df_ancestry['suburb'].str.contains(suburb)]['Ancestry Cluster'][0])

In [ ]:
for suburb in df_complete[pd.isnull(df_complete['Ancestry Cluster'])]['suburb']:
    if(len(df_ancestry.loc[df_ancestry['suburb'].str.contains(suburb)]['Ancestry Cluster'])>0):
        print(df_complete[df_complete['suburb']==suburb]['Ancestry Cluster'])
        print("will get cluster:",df_ancestry.loc[df_ancestry['suburb'].str.contains(suburb)]['Ancestry Cluster'][0])
        df_complete[df_complete['suburb']==suburb]['Ancestry Cluster']=df_ancestry.loc[df_ancestry['suburb'].str.contains(suburb)]['Ancestry Cluster'][0]

In [ ]:
df_complete

In [ ]:
suburb = 'seddon'
df_ancestry[df_ancestry['suburb'].str.contains(suburb)]

In [ ]:
df_ancestry[df_ancestry['suburb'].str.contains('melbourne')]['Ancestry Cluster']
                                                                        

In [ ]:
#fuzzy match??
import difflib
df_ancestry2=df_ancestry
df_melbourne_merged2=df_melbourne_merged

df_ancestry2['suburb'] = df_ancestry2['suburb'].apply(lambda x: difflib.get_close_matches(x, df_melbourne_merged2['suburb'])[0])
df_melbourne_merged2.merge(df_ancestry2)

# Assign age and income brackets


In [ ]:
#define a function which takes a suburb name and scrapes the web for median age and household income statistics
def get_age_income(state,suburb):
    
    #Format the url
    url='https://propertydata.realestateview.com.au/propertydata/suburb-profile/'+state+'/'
    suburb = suburb.split()
    i=0
    for i in suburb:
        url=url+i+"+"

    #get the page text for the suburb
    page = requests.get(url).text
    soup = BeautifulSoup(page, 'lxml')
    
    #find all the important elements
    sp = soup.find_all('b')
    
    #get the median age and income for the suburb
    for element in sp:
        if element.text=='Median Age':
            age = element.next_sibling.next_sibling.translate({ord(i): None for i in '\n\t'})
        if element.text=='Weekly Household Income':
            income = element.next_sibling.next_sibling.translate({ord(i): None for i in '\n\t'})
        
    return age, income

In [ ]:
get_age_income('victoria','bright')